In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"

In [2]:
# from __future__ import print_function
# import argparse
import torch
import torch.utils.data
import torch.nn as nn 
import torch.optim as optim
from torch.autograd import Variable  # change later
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
from torch.nn.parameter import Parameter # ?

from functools import reduce
from torch.utils.data import DataLoader, TensorDataset


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

##### Dataset

In [4]:
url = 'https://raw.githubusercontent.com/chl8856/DeepHit/master/sample%20data/SYNTHETIC/synthetic_comprisk.csv'
dataset = pd.read_csv(url)
dataset.drop(['true_time', 'true_label'], axis=1, inplace=True)
dataset.head()

,time,label,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12
0,0,0,-0.440500,-0.035066,-0.025341,-0.029775,-0.55787,-0.15355,0.56819,-0.15432,-0.250230,0.33915,0.70388,0.28174
1,1,0,0.015579,-0.846080,0.487530,0.651930,0.20099,-0.11238,-1.39630,-0.18874,-0.300010,-0.24032,-0.38533,-1.02450
2,34,2,0.446490,1.641000,-1.745000,0.317950,-1.14060,0.36560,0.28110,-0.58253,-1.690700,1.20220,-0.51920,1.78400
3,9,0,0.629460,-0.615750,-0.323450,-0.900200,0.45360,-0.61992,2.16240,0.19875,-1.119600,-2.73210,-0.25673,-0.81836
4,2,0,1.249800,-0.185610,-0.183780,-0.981080,-0.01499,-0.14437,-1.25290,-0.58432,-0.090523,0.93692,1.07490,0.79117


In [31]:
# Change values of outcome
print(dataset['label'])
dataset['label'].replace({2:1}, inplace=True)
# print(dataset['label'])

0        0
1        0
2        1
3        0
4        0
        ..
29995    1
29996    1
29997    0
29998    0
29999    0
Name: label, Length: 30000, dtype: int64


##### Train - test - validation split


In [6]:
get_x = lambda df: (df
                    .drop(columns=['time', 'label'])
                    .values.astype('float32'))

df_test = dataset.sample(frac=0.2)
df_train = dataset.drop(df_test.index)
# df_val = df_train.sample(frac=0.2)
# df_train = df_train.drop(df_val.index)

X_train = get_x(df_train)
X_test = get_x(df_test)

Y_train = df_train[['label', 'time']].to_numpy()
Y_test = df_test[['label', 'time']].to_numpy()


In [7]:
D_in, H, D_out = X_train.shape[1], 3, 5    # D_out 32 ?
batch_size = 32
num_time_units = 10 # 24 month?
time_bin = 30   # 30?
n_epochs = 1
learning_rate = 1e-3


In [8]:
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
# X_train, y_train, E_train = np.array(X_train), np.array(y_train), np.array(E_train)
# X_test, y_test, E_test = np.array(X_test), np.array(y_test), np.array(E_test)
train_dataset = TensorDataset(torch.from_numpy(X_train).float().to(device), torch.from_numpy(Y_train[:,1]).float().to(device), torch.from_numpy(Y_train[:,0]).float().to(device))
test_dataset = TensorDataset(torch.from_numpy(X_test).float().to(device), torch.from_numpy(Y_test[:,1]).float().to(device), torch.from_numpy(Y_test[:,0]).float().to(device))

In [25]:

class Net_regular(nn.Module):
    def __init__(self, n_feature, num_layers, node, dropout, drop_factor = 1):
        super(Net_regular, self).__init__()
        # input layer
        layers = [nn.Linear(n_feature, node),
                  nn.BatchNorm1d(node),
                  nn.ReLU()]
        # hidden layers
        node_temp = node
        for i in range(0, num_layers):
            node_temp0 = max(4, int(node_temp / (drop_factor**i)))
            node_temp1 = max(4, int(node_temp0 / drop_factor))
            layers += [nn.Linear(node_temp0, node_temp1),
                       nn.BatchNorm1d(node_temp1),
                       nn.ReLU(),
                       nn.Dropout(p=dropout)]
        layers += [nn.ReLU()]
        # output layer
        layers += [nn.Linear(node_temp1, 1)]
        self.seq = nn.Sequential(*layers) 

    def forward(self, inputs):
        return self.seq(inputs)
    

##### Function for C-index

In [10]:
def onePair(x0, x1):
    c = np.log(2.)
    m = nn.LogSigmoid() 
    return 1 + m(x1-x0) / c
  
def rank_loss(pred, obs, delta):
    N = pred.size(0)
    allPairs = onePair(pred.view(N,1), pred.view(1,N))

    temp0 = obs.view(1, N) - obs.view(N, 1)
    # indices based on obs time
    temp1 = temp0>0
    # indices of event-event or event-censor pair
    temp2 = delta.view(1, N) + delta.view(N, 1)
    temp3 = temp2>0
    # indices of events
    temp4 = delta.view(N, 1) * torch.ones(1, N, device = device)
    # selected indices
    final_ind = temp1 * temp3 * temp4
    out = allPairs * final_ind
    return out.sum() / final_ind.sum()

def mse_loss(pred,  obs, delta):
    mse = delta*((pred - obs) ** 2)

    ind = pred < obs
    delta0 = 1 - delta
    p = ind * delta0 * (obs - pred)**2 
    return mse.mean(), p.mean()

In [11]:
# model = survdl(D_in, H, D_out, num_time_units).to(device)

# optimizer = optim.Adam(model.parameters(), lr = learning_rate)


##### Training and evaluating

In [29]:
lambda1 = 1
lambda2 = 0.2
torch.manual_seed(1)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

# model = survdl( D_in, H, D_out, num_time_units)
model = Net_regular(n_feature=D_in, num_layers = 1, node=128, dropout=0.5)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-8)

# Trianing
epoch_loss_train = []
for e in range(1, n_epochs+1):
    model.train()
    for X_train_batch, y_train_batch, E_train_batch in train_loader:
        optimizer.zero_grad()

        y_train_pred = model(X_train_batch)
        mseloss, penaltyloss = mse_loss(y_train_pred, y_train_batch.unsqueeze(1), E_train_batch.unsqueeze(1))
        rankloss = rank_loss(y_train_pred, y_train_batch.unsqueeze(1), E_train_batch.unsqueeze(1))
        train_loss = mseloss + lambda1*penaltyloss - lambda2*rankloss

        train_loss.backward()
        optimizer.step()

# Predicting train
train_loader1 = DataLoader(dataset=train_dataset, batch_size=len(train_dataset))
y_pred_list0 = []
with torch.no_grad():
    model.eval()
    for X_batch, y_batch, E_batch in train_loader1:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_pred_list0.append(y_test_pred.cpu().numpy())
y_pred_list0 = [a.squeeze().tolist() for a in y_pred_list0]
y_pred_list0 = sum(y_pred_list0, [])

# Predicting test
with torch.no_grad():
    model.train() 
    result = []
    for _ in range(100):   #T 
        y_pred_list = [] 
        for X_batch, y_batch, E_batch in test_loader:
            y_test_pred = model(X_batch)
            y_pred_list.append(y_test_pred.cpu().numpy())
            y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
            y_pred_list = sum(y_pred_list, [])
        result.append(y_pred_list)

    result = np.array(result)
    y_test_pred_mean = result.mean(axis=0).reshape(-1,)
    y_test_pred_sd = result.std(axis=0).reshape(-1,)
    y_pred_list_upper = y_test_pred_mean + 1.96*y_test_pred_sd
    y_pred_list_lower = y_test_pred_mean - 1.96*y_test_pred_sd


In [30]:
import lifelines

print("Train C-index: ", lifelines.utils.concordance_index(Y_train[:,1], 
                                                           np.exp(y_pred_list0),
                                                           Y_train[:,0]))
print("Test C-index: ", lifelines.utils.concordance_index(Y_test[:,1], 
                                                          np.exp(y_pred_list),
                                                          Y_test[:,0]))
# print("")
# print("Train MSE: ", mean_squared_error(np.log(train_df["FT"]), y_train_pred))
# print("Test MSE: ", mean_squared_error(np.log(test_df["FT"]), y_test_pred))

Train C-index:  0.7289589468440526
Test C-index:  0.7216766462628683


In [21]:
len(y_pred_list)

6000

In [ ]:
!pip install lifelines
